In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data

%matplotlib inline

# import data

In [2]:
# a = np.loadtxt('./GSE107011_Processed_data_TPM.txt', skiprows = 1,dtype=str, unpack = True)
# a = np.genfromtxt('./GSE107011_Processed_data_TPM.txt', names = True, dtype = None, unpack = False, encoding='ascii')
# Todo: read the head(sample name) directly as index
a = np.loadtxt('./GSE107011_Processed_data_TPM.txt', dtype=float, skiprows=1, usecols=(range(1,128)), unpack=True)
# data = torch.from_numpy(a)
# data.size()

## Data preparation

In [3]:
class Dataset(data.Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
#         'Denotes the total number of samples'
        return len(self.data)
    
    def __getitem__(self, index):
#         'Generates one sample of data'
        return torch.from_numpy(self.data[index])

dataset = Dataset(a)
train_loader = data.DataLoader(dataset=dataset, batch_size=10, shuffle=False)

# Simple Linear Autoencoder

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: True 


## Stacked Autoencoder

In [5]:
def train_model(model,loss_fn,data_loader=None,epochs=1,optimizer=None):
    model.train()
    for epoch in range(epochs):
        i = 0
        for batch_data in train_loader:
            i+=1
            batch_data = batch_data.type('torch.FloatTensor')
            data = batch_data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, data)
            loss.backward()
            optimizer.step()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(data_loader.dataset),
                100. * i / len(data_loader), loss.data.item()))

In [6]:
class DeepAutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(DeepAutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 2048),
#             nn.ReLU(True),
#             nn.Linear(16384, 8192),
#             nn.ReLU(True), 
            nn.Linear(2048,1024),
#             nn.ReLU(True), 
            nn.Linear(1024, encoding_dim), 
#             nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 1024),
#             nn.ReLU(True),
            nn.Linear(1024,2048),
#             nn.ReLU(True), 
#             nn.Linear(8192,16384),
#             nn.ReLU(True), 
            nn.Linear(2048, input_dim),
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
input_dim = len(a[1])
encoding_dim = 512

model = DeepAutoEncoder(input_dim, encoding_dim)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.MSELoss()

In [8]:
train_model(model, loss_fn,data_loader=train_loader,epochs=100,optimizer=optimizer)

Train Epoch: 0 [10/127 (8%)]	Loss: 65536.570312
Train Epoch: 0 [20/127 (15%)]	Loss: 120474.703125
Train Epoch: 0 [30/127 (23%)]	Loss: 472959.500000
Train Epoch: 0 [40/127 (31%)]	Loss: 1081175.625000
Train Epoch: 0 [50/127 (38%)]	Loss: 236201.078125
Train Epoch: 0 [60/127 (46%)]	Loss: 223420.218750
Train Epoch: 0 [70/127 (54%)]	Loss: 3096361.000000
Train Epoch: 0 [80/127 (62%)]	Loss: 187904.390625
Train Epoch: 0 [90/127 (69%)]	Loss: 206500.078125
Train Epoch: 0 [100/127 (77%)]	Loss: 414051.250000
Train Epoch: 0 [110/127 (85%)]	Loss: 144082.890625
Train Epoch: 0 [120/127 (92%)]	Loss: 119869.398438
Train Epoch: 0 [91/127 (100%)]	Loss: 239062.859375
Train Epoch: 1 [10/127 (8%)]	Loss: 94807.015625
Train Epoch: 1 [20/127 (15%)]	Loss: 153219.796875
Train Epoch: 1 [30/127 (23%)]	Loss: 167711.781250
Train Epoch: 1 [40/127 (31%)]	Loss: 557839.125000
Train Epoch: 1 [50/127 (38%)]	Loss: 665184.187500
Train Epoch: 1 [60/127 (46%)]	Loss: 362702.375000
Train Epoch: 1 [70/127 (54%)]	Loss: 322370.03125

Train Epoch: 12 [110/127 (85%)]	Loss: 6600.961914
Train Epoch: 12 [120/127 (92%)]	Loss: 14354.868164
Train Epoch: 12 [91/127 (100%)]	Loss: 10869.638672
Train Epoch: 13 [10/127 (8%)]	Loss: 7366.998047
Train Epoch: 13 [20/127 (15%)]	Loss: 9972.127930
Train Epoch: 13 [30/127 (23%)]	Loss: 13668.531250
Train Epoch: 13 [40/127 (31%)]	Loss: 7006.096191
Train Epoch: 13 [50/127 (38%)]	Loss: 17942.273438
Train Epoch: 13 [60/127 (46%)]	Loss: 10716.803711
Train Epoch: 13 [70/127 (54%)]	Loss: 5355.441895
Train Epoch: 13 [80/127 (62%)]	Loss: 9044.578125
Train Epoch: 13 [90/127 (69%)]	Loss: 11409.539062
Train Epoch: 13 [100/127 (77%)]	Loss: 11757.286133
Train Epoch: 13 [110/127 (85%)]	Loss: 6361.396484
Train Epoch: 13 [120/127 (92%)]	Loss: 13875.040039
Train Epoch: 13 [91/127 (100%)]	Loss: 10559.018555
Train Epoch: 14 [10/127 (8%)]	Loss: 7354.509277
Train Epoch: 14 [20/127 (15%)]	Loss: 9725.359375
Train Epoch: 14 [30/127 (23%)]	Loss: 13146.634766
Train Epoch: 14 [40/127 (31%)]	Loss: 6894.958496
Train

Train Epoch: 25 [80/127 (62%)]	Loss: 6254.717773
Train Epoch: 25 [90/127 (69%)]	Loss: 8730.887695
Train Epoch: 25 [100/127 (77%)]	Loss: 7203.932617
Train Epoch: 25 [110/127 (85%)]	Loss: 4456.383301
Train Epoch: 25 [120/127 (92%)]	Loss: 9860.920898
Train Epoch: 25 [91/127 (100%)]	Loss: 6080.288086
Train Epoch: 26 [10/127 (8%)]	Loss: 5483.820312
Train Epoch: 26 [20/127 (15%)]	Loss: 6263.724609
Train Epoch: 26 [30/127 (23%)]	Loss: 10702.868164
Train Epoch: 26 [40/127 (31%)]	Loss: 5500.672852
Train Epoch: 26 [50/127 (38%)]	Loss: 9464.204102
Train Epoch: 26 [60/127 (46%)]	Loss: 7557.944824
Train Epoch: 26 [70/127 (54%)]	Loss: 2682.848633
Train Epoch: 26 [80/127 (62%)]	Loss: 5701.419434
Train Epoch: 26 [90/127 (69%)]	Loss: 8243.265625
Train Epoch: 26 [100/127 (77%)]	Loss: 6753.374512
Train Epoch: 26 [110/127 (85%)]	Loss: 4706.873535
Train Epoch: 26 [120/127 (92%)]	Loss: 9670.175781
Train Epoch: 26 [91/127 (100%)]	Loss: 5482.759766
Train Epoch: 27 [10/127 (8%)]	Loss: 4891.189941
Train Epoch: 

Train Epoch: 38 [50/127 (38%)]	Loss: 7549.965820
Train Epoch: 38 [60/127 (46%)]	Loss: 4728.477539
Train Epoch: 38 [70/127 (54%)]	Loss: 2852.326904
Train Epoch: 38 [80/127 (62%)]	Loss: 6144.209961
Train Epoch: 38 [90/127 (69%)]	Loss: 6195.857422
Train Epoch: 38 [100/127 (77%)]	Loss: 5711.566406
Train Epoch: 38 [110/127 (85%)]	Loss: 4316.968262
Train Epoch: 38 [120/127 (92%)]	Loss: 6497.081055
Train Epoch: 38 [91/127 (100%)]	Loss: 2567.860352
Train Epoch: 39 [10/127 (8%)]	Loss: 4027.363770
Train Epoch: 39 [20/127 (15%)]	Loss: 6045.952637
Train Epoch: 39 [30/127 (23%)]	Loss: 7254.248047
Train Epoch: 39 [40/127 (31%)]	Loss: 3963.534180
Train Epoch: 39 [50/127 (38%)]	Loss: 6840.445801
Train Epoch: 39 [60/127 (46%)]	Loss: 4191.779297
Train Epoch: 39 [70/127 (54%)]	Loss: 2500.123779
Train Epoch: 39 [80/127 (62%)]	Loss: 5087.637695
Train Epoch: 39 [90/127 (69%)]	Loss: 5870.267090
Train Epoch: 39 [100/127 (77%)]	Loss: 5779.394043
Train Epoch: 39 [110/127 (85%)]	Loss: 4371.297852
Train Epoch: 39

Train Epoch: 51 [30/127 (23%)]	Loss: 8997.510742
Train Epoch: 51 [40/127 (31%)]	Loss: 6716.694824
Train Epoch: 51 [50/127 (38%)]	Loss: 6986.237305
Train Epoch: 51 [60/127 (46%)]	Loss: 3263.432129
Train Epoch: 51 [70/127 (54%)]	Loss: 5005.606934
Train Epoch: 51 [80/127 (62%)]	Loss: 11728.758789
Train Epoch: 51 [90/127 (69%)]	Loss: 9244.975586
Train Epoch: 51 [100/127 (77%)]	Loss: 6743.443359
Train Epoch: 51 [110/127 (85%)]	Loss: 4839.214844
Train Epoch: 51 [120/127 (92%)]	Loss: 9039.215820
Train Epoch: 51 [91/127 (100%)]	Loss: 8569.252930
Train Epoch: 52 [10/127 (8%)]	Loss: 12709.801758
Train Epoch: 52 [20/127 (15%)]	Loss: 10046.600586
Train Epoch: 52 [30/127 (23%)]	Loss: 9265.082031
Train Epoch: 52 [40/127 (31%)]	Loss: 8723.458008
Train Epoch: 52 [50/127 (38%)]	Loss: 8781.131836
Train Epoch: 52 [60/127 (46%)]	Loss: 4876.080078
Train Epoch: 52 [70/127 (54%)]	Loss: 4118.539062
Train Epoch: 52 [80/127 (62%)]	Loss: 9200.944336
Train Epoch: 52 [90/127 (69%)]	Loss: 8481.872070
Train Epoch: 5

Train Epoch: 64 [10/127 (8%)]	Loss: 9001.969727
Train Epoch: 64 [20/127 (15%)]	Loss: 10379.287109
Train Epoch: 64 [30/127 (23%)]	Loss: 6688.828613
Train Epoch: 64 [40/127 (31%)]	Loss: 8633.866211
Train Epoch: 64 [50/127 (38%)]	Loss: 16047.791016
Train Epoch: 64 [60/127 (46%)]	Loss: 9460.733398
Train Epoch: 64 [70/127 (54%)]	Loss: 6124.352539
Train Epoch: 64 [80/127 (62%)]	Loss: 7174.817383
Train Epoch: 64 [90/127 (69%)]	Loss: 5358.419922
Train Epoch: 64 [100/127 (77%)]	Loss: 5665.178711
Train Epoch: 64 [110/127 (85%)]	Loss: 3802.990723
Train Epoch: 64 [120/127 (92%)]	Loss: 6729.059082
Train Epoch: 64 [91/127 (100%)]	Loss: 4867.863281
Train Epoch: 65 [10/127 (8%)]	Loss: 8398.829102
Train Epoch: 65 [20/127 (15%)]	Loss: 6563.293457
Train Epoch: 65 [30/127 (23%)]	Loss: 11523.066406
Train Epoch: 65 [40/127 (31%)]	Loss: 4830.090332
Train Epoch: 65 [50/127 (38%)]	Loss: 9330.286133
Train Epoch: 65 [60/127 (46%)]	Loss: 6736.979492
Train Epoch: 65 [70/127 (54%)]	Loss: 5009.837891
Train Epoch: 65

Train Epoch: 76 [110/127 (85%)]	Loss: 7900.870117
Train Epoch: 76 [120/127 (92%)]	Loss: 9982.335938
Train Epoch: 76 [91/127 (100%)]	Loss: 3730.280762
Train Epoch: 77 [10/127 (8%)]	Loss: 5076.791016
Train Epoch: 77 [20/127 (15%)]	Loss: 13558.093750
Train Epoch: 77 [30/127 (23%)]	Loss: 7299.111328
Train Epoch: 77 [40/127 (31%)]	Loss: 4622.819336
Train Epoch: 77 [50/127 (38%)]	Loss: 7976.703125
Train Epoch: 77 [60/127 (46%)]	Loss: 3389.622314
Train Epoch: 77 [70/127 (54%)]	Loss: 1920.732056
Train Epoch: 77 [80/127 (62%)]	Loss: 10502.270508
Train Epoch: 77 [90/127 (69%)]	Loss: 6223.332520
Train Epoch: 77 [100/127 (77%)]	Loss: 7994.521973
Train Epoch: 77 [110/127 (85%)]	Loss: 4466.436523
Train Epoch: 77 [120/127 (92%)]	Loss: 8611.812500
Train Epoch: 77 [91/127 (100%)]	Loss: 3384.630371
Train Epoch: 78 [10/127 (8%)]	Loss: 8006.552246
Train Epoch: 78 [20/127 (15%)]	Loss: 8785.662109
Train Epoch: 78 [30/127 (23%)]	Loss: 6785.547363
Train Epoch: 78 [40/127 (31%)]	Loss: 4472.541504
Train Epoch: 

Train Epoch: 89 [90/127 (69%)]	Loss: 3202.533203
Train Epoch: 89 [100/127 (77%)]	Loss: 4909.957031
Train Epoch: 89 [110/127 (85%)]	Loss: 3934.675293
Train Epoch: 89 [120/127 (92%)]	Loss: 3809.381836
Train Epoch: 89 [91/127 (100%)]	Loss: 2350.421631
Train Epoch: 90 [10/127 (8%)]	Loss: 3605.983887
Train Epoch: 90 [20/127 (15%)]	Loss: 3410.208008
Train Epoch: 90 [30/127 (23%)]	Loss: 4177.762207
Train Epoch: 90 [40/127 (31%)]	Loss: 4030.790283
Train Epoch: 90 [50/127 (38%)]	Loss: 6340.895508
Train Epoch: 90 [60/127 (46%)]	Loss: 3641.039062
Train Epoch: 90 [70/127 (54%)]	Loss: 2775.219727
Train Epoch: 90 [80/127 (62%)]	Loss: 4038.131592
Train Epoch: 90 [90/127 (69%)]	Loss: 2836.807617
Train Epoch: 90 [100/127 (77%)]	Loss: 3632.864990
Train Epoch: 90 [110/127 (85%)]	Loss: 3047.021729
Train Epoch: 90 [120/127 (92%)]	Loss: 2655.961914
Train Epoch: 90 [91/127 (100%)]	Loss: 3689.646973
Train Epoch: 91 [10/127 (8%)]	Loss: 3382.260498
Train Epoch: 91 [20/127 (15%)]	Loss: 3472.479980
Train Epoch: 9

In [9]:
test_data0 = torch.from_numpy(a[0])
test_data0 = test_data0.type('torch.FloatTensor')
test_data0 = test_data0.to(device)
encoded_data0 = model.module.encoder(test_data0)

In [10]:
test_data1 = torch.from_numpy(a[1])
test_data1 = test_data1.type('torch.FloatTensor')
test_data1 = test_data1.to(device)
encoded_data1 = model.module.encoder(test_data1)

In [11]:
test_data3 = torch.from_numpy(a[0]+a[1])
test_data3 = test_data3.type('torch.FloatTensor')
test_data3 = test_data3.to(device)
encoded_data3 = model.module.encoder(test_data3)

In [12]:
encoded_data3 - encoded_data0 -encoded_data1

tensor([ 6.3477e-03, -2.8503e-02, -3.5980e-02, -4.2717e-02, -1.0071e-03,
         2.7954e-02, -3.7109e-02, -5.6534e-03, -1.9974e-02, -3.1555e-02,
         4.3396e-02,  4.6177e-02, -2.6779e-02,  7.5684e-03, -1.4038e-02,
        -7.6294e-04, -2.4780e-02,  1.8066e-02, -2.4246e-02, -4.3335e-03,
         2.0473e-02, -2.2644e-02,  4.2419e-03, -2.8778e-02, -1.4475e-02,
        -2.7100e-02, -5.7068e-03, -3.1738e-03, -8.2397e-04, -5.6152e-03,
        -1.7845e-02, -8.6060e-03,  2.5696e-02,  8.9188e-03, -1.7813e-02,
         2.3499e-02, -1.3672e-02, -3.2166e-02, -4.0161e-02, -5.9814e-03,
        -2.6245e-03, -7.0229e-03, -2.8076e-03, -2.7618e-02,  2.6947e-02,
        -8.1177e-03,  3.2013e-02, -9.7046e-03,  4.6326e-02, -5.8594e-03,
         2.5757e-02,  1.1658e-02,  6.9275e-03, -1.8311e-04, -3.2883e-02,
         2.5513e-02,  5.8136e-03,  7.9346e-04,  7.9117e-03,  1.6907e-02,
         1.4038e-02,  8.3618e-03,  4.3945e-03,  3.4790e-03,  9.2163e-03,
        -2.9816e-02, -6.3477e-02,  2.5513e-02, -6.4

In [13]:
test_data0

tensor([ 0.2143,  4.7596,  0.0000,  ...,  0.0000,  1.6853, 53.6174],
       device='cuda:0')

In [14]:
encoded_data0

tensor([-7.0996e+02,  1.3838e+03, -2.7657e+02,  1.7381e+02, -8.0764e+02,
        -4.4667e+02,  1.6538e+03, -3.3959e+02,  3.0095e+01, -1.0127e+03,
        -8.0965e+02, -2.2723e+02, -1.1859e+02,  1.9088e+03,  7.7771e+02,
         1.3478e+00, -9.8462e+02, -4.6163e+02,  3.8760e+02,  3.4913e+02,
        -5.3808e+02,  1.0432e+03, -7.9333e+02, -5.8211e+01, -5.1734e+02,
        -1.0182e+02,  3.0921e+02, -8.5020e+02,  7.4049e+02, -4.1386e+02,
        -4.8993e+02,  6.4466e+02,  7.2970e+02, -4.6636e+02,  1.0676e+02,
         8.7692e+02,  2.7649e+03, -5.6593e+02,  9.3169e+02, -4.4889e+02,
        -1.3800e+02, -2.5234e+02,  1.2548e+03, -9.1330e+02,  7.3846e+02,
        -1.1178e+03, -2.1298e+02,  3.7805e+02,  6.8760e+01,  1.6672e+03,
        -1.3964e+03,  1.7100e+02, -3.6106e+01, -1.1435e+02,  8.4892e+01,
         7.0213e+02, -7.0040e+00, -1.0834e+03,  3.9108e+02,  9.3170e+02,
        -1.6041e+03, -7.5843e+02,  4.2016e+02,  1.1246e+03,  4.6138e+02,
        -2.6019e+02,  3.0228e+03,  2.2010e+02,  1.7